# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# General libraries
import re
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import  create_engine
# NLTK libraries
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
# Scikit-learn libraries
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
import pickle

In [2]:
# load data from database
engine = create_engine('sqlite:///etl_pipeline.db')
df = pd.read_sql_table('etl_pipeline', con=engine) 
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
df['message'][1]

'Is the Hurricane over or is it not over'

In [3]:
df['related'].unique()

array([1, 0, 2])

In [86]:
df.child_alone.value_counts()

0    26028
Name: child_alone, dtype: int64

In [5]:
# Clean up columns that has value of 2
df = df[df.related != 2]

In [6]:
# Define feature and target variables X and Y
X = df.message.values
Y = df[df.columns[4:]] # equivalent to Y = df.iloc[:, 4:] 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
# Building the model
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf', TfidfTransformer()), 
    ('clf', MultiOutputClassifier(RandomForestClassifier())) #KNeighborsClassifier()
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [11]:
type(X_train) #.dtype

numpy.ndarray

In [12]:
# Fit it on a subset of data first
#'''
X_train_subset = X_train[0:10]
y_train_subset = y_train[0:10]
pipeline.fit(X_train_subset, y_train_subset) 
#'''

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x12cb12268>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [13]:
X_train_subset

array(['Since then, more than 9 million people have been directly affected by the cyclone which caused extensive damages to houses, water and sanitation infrastructures, harvests, food stocks and livelihoods, also killing livestock and destroying fishing material and other income generating assets.',
       'Rangelands have benefited from the favourable rains, and livestock condition is generally good at present.',
       'good evening I need help water medicaments wc paper tent COLEMAN thanks rue ALIX ROI IMPASSE CHAMPAGNE ',
       'You must know that we have organizations for the environment. How can we help you to start a project? ',
       'It added that a drop in hydropower supplies from the Cahora Bassa dam to South Africa has meant that South Africa must use more expensive electriciy from its thermal power stations.',
       'Haitians in the U.S. are trying desperately to contact their loved ones on the earthquake ravaged island http bit.ly 8qMxXz',
       'Figure 1 illustrates

In [25]:
%%timeit
pipeline.fit(X_train, y_train) 

KeyboardInterrupt: 

In [26]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [27]:
categories = Y.columns.values
print(categories)
#y_test_df = pd.DataFrame(y_test, columns=categories)
#y_pred_df = pd.DataFrame(y_pred, columns=categories)

['related' 'request' 'offer' 'aid_related' 'medical_help'
 'medical_products' 'search_and_rescue' 'security' 'military'
 'child_alone' 'water' 'food' 'shelter' 'clothing' 'money'
 'missing_people' 'refugees' 'death' 'other_aid' 'infrastructure_related'
 'transport' 'buildings' 'electricity' 'tools' 'hospitals' 'shops'
 'aid_centers' 'other_infrastructure' 'weather_related' 'floods' 'storm'
 'fire' 'earthquake' 'cold' 'other_weather' 'direct_report']


In [29]:
for i, c in enumerate(categories):
    print(c)
    print(classification_report(y_test.iloc[:,i] , y_pred[:,i]))

related
              precision    recall  f1-score   support

           0       0.71      0.24      0.36      1546
           1       0.80      0.97      0.88      4961

    accuracy                           0.80      6507
   macro avg       0.76      0.61      0.62      6507
weighted avg       0.78      0.80      0.76      6507

request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5378
           1       0.90      0.42      0.57      1129

    accuracy                           0.89      6507
   macro avg       0.89      0.71      0.76      6507
weighted avg       0.89      0.89      0.87      6507

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6480
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      650

/Users/lin/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6425
           1       0.00      0.00      0.00        82

    accuracy                           0.99      6507
   macro avg       0.49      0.50      0.50      6507
weighted avg       0.97      0.99      0.98      6507

other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6227
           1       0.00      0.00      0.00       280

    accuracy                           0.96      6507
   macro avg       0.48      0.50      0.49      6507
weighted avg       0.92      0.96      0.94      6507

weather_related
              precision    recall  f1-score   support

           0       0.86      0.97      0.91      4693
           1       0.87      0.60      0.71      1814

    accuracy                           0.87      6507
   macro avg       0.87      0.78      0.81      6507
weighted avg       0.87      0.87    

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
#multilabel_confusion_matrix(y_test,y_pred)

### 6. Improve your model
Use grid search to find better parameters. 

In [30]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x12cb12268>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x12cb12268>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_i

In [31]:
parameters = {
    'vect__ngram_range':[(1,1),(1,2),(1,3)],
    #'vect__binary':[True, False],
    #'vect__strip_accents':['unicode', None],
    'tfidf__norm':['l1', 'l2']
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=3)

In [32]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed: 445.3min finished


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x12cb12268>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=4,
             param_grid={'tfidf__norm': ['l1', 'l2'],
                         'vect__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             verbose=3)

In [34]:
print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'tfidf__norm': 'l1', 'vect__ngram_range': (1, 3)}


In [35]:
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [37]:
for i, c in enumerate(categories):
    print(c)
    print(classification_report(y_test.iloc[:,i] , y_pred[:,i])) #target_names=Y.columns

related
              precision    recall  f1-score   support

           0       0.71      0.26      0.38      1546
           1       0.81      0.97      0.88      4961

    accuracy                           0.80      6507
   macro avg       0.76      0.61      0.63      6507
weighted avg       0.78      0.80      0.76      6507

request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5378
           1       0.87      0.42      0.57      1129

    accuracy                           0.89      6507
   macro avg       0.88      0.71      0.75      6507
weighted avg       0.89      0.89      0.87      6507

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6480
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      650

/Users/lin/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6436
           1       0.00      0.00      0.00        71

    accuracy                           0.99      6507
   macro avg       0.49      0.50      0.50      6507
weighted avg       0.98      0.99      0.98      6507

shops
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6473
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      0.99      0.99      6507

aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6425
           1       0.00      0.00      0.00        82

    accuracy                           0.99      6507
   macro avg       0.49      0.50      0.50      6507
weighted avg       0.97      0.99      0.98      6507

o

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [38]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [39]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [40]:
def tokenize(text):
    '''
    This function first normalizes the plain texts, detects URLs in the messages, replace those URLs with a place holder;
    It further tokenizes words and removes stop words;
    After maps and lemmatizes different versions of same words to their root forms, 
    the function finally returns the clean tokens.
    '''
    detected_urls = re.findall(url_regex, text)
    text = text.lower()
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    word_tokens = word_tokenize(text)
    
    words = [w for w in word_tokens if w not in stopwords.words('english')]
    
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in words:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [59]:
# Testing tokenize() function
tokenize('Since then, more than 9 million people have been directly affected by the cyclone which caused extensive damages to houses, water and sanitation infrastructures, harvests, food stocks and livelihoods, also killing livestock and destroying fishing material and other income generating assets.')

['since',
 ',',
 '9',
 'million',
 'people',
 'directly',
 'affected',
 'cyclone',
 'caused',
 'extensive',
 'damage',
 'house',
 ',',
 'water',
 'sanitation',
 'infrastructure',
 ',',
 'harvest',
 ',',
 'food',
 'stock',
 'livelihood',
 ',',
 'also',
 'killing',
 'livestock',
 'destroying',
 'fishing',
 'material',
 'income',
 'generating',
 'asset',
 '.']

In [65]:
# Refitting
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf', TfidfTransformer()), 
    ('clf', MultiOutputClassifier(RandomForestClassifier())) 
])

In [66]:
# Grid search
parameters = {
    'vect__ngram_range':[(1,1),(1,2),(1,3)],
    'tfidf__norm':['l1', 'l2']
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=3)

In [67]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed: 135.4min finished


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x12daafc80>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=4,
             param_grid={'tfidf__norm': ['l1', 'l2'],
                         'vect__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             verbose=3)

In [68]:
print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'tfidf__norm': 'l1', 'vect__ngram_range': (1, 3)}


In [70]:
y_pred = cv.predict(X_test)

In [50]:
# Reevaluation
for i, c in enumerate(categories):
    print(c)
    print(classification_report(y_test.iloc[:,i] , y_pred[:,i])) #target_names=Y.columns

related
              precision    recall  f1-score   support

           0       0.33      0.00      0.00      1546
           1       0.76      1.00      0.87      4961

    accuracy                           0.76      6507
   macro avg       0.55      0.50      0.43      6507
weighted avg       0.66      0.76      0.66      6507

request
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      5378
           1       0.67      0.00      0.00      1129

    accuracy                           0.83      6507
   macro avg       0.75      0.50      0.45      6507
weighted avg       0.80      0.83      0.75      6507

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6480
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      650

/Users/lin/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6436
           1       0.00      0.00      0.00        71

    accuracy                           0.99      6507
   macro avg       0.49      0.50      0.50      6507
weighted avg       0.98      0.99      0.98      6507

shops
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6473
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      0.99      0.99      6507

aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6425
           1       0.00      0.00      0.00        82

    accuracy                           0.99      6507
   macro avg       0.49      0.50      0.50      6507
weighted avg       0.97      0.99      0.98      6507

o

### 9. Export your model as a pickle file

In [53]:
# Save the model to disk
pickle.dump(pipeline, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

#### Imbalanced Data
It can be seen that the count for different categories are imbalanced, this will normally require us to look carefully at any sources of bias during data collections are accounted for. 